In [1]:
import numpy as np
import pandas as pd 
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from category_encoders.count import CountEncoder
import gc

NUM_FOLDS=5
RANDOM_STATE = 800


#     ░░░░░░░░░░░░        ▒▒▓▓██▓▓▓▓▓▓▓▓▓▓██▓▓▓▓▓▓    ░░░░░░░░░░░░░░        
#           ░░░░░░░░░░░░  ██████░░░░░░░░░░░░░░░░  ░░░░░░████░░░░░░░░░░░░░░        
#           ░░░░░░░░░░████  ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░████░░░░░░░░          
#                   ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░    ██                
#               ░░██    ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██              
#             ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██          
#               ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██            
#             ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██          
#           ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓        
#           ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██        
#           ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██        
#         ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██      
#         ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░    ░░██      
#         ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░  ░░    ░░██      
#         ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██      
#       ▓▓░░░░░░████░░▓▓▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓░░░░░░░░░░▒▒▓▓    
#       ██░░▒▒▓▓▒▒▒▒██▒▒▒▒██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██░░░░░░░░░░░░██    
#       ██▒▒░░██▒▒▒▒▒▒▒▒▒▒██░░░░░░░░░░░░░░▓▓░░░░░░░░░░░░░░░░░░██░░░░░░░░░░░░██    
#       ██░░░░▒▒██▒▒▒▒▒▒██░░░░░░░░░░░░░░░░██░░░░░░░░░░░░░░░░░░██░░░░░░░░░░░░██    
#       ██░░░░░░░░██████▒▒░░░░░░░░░░░░░░██░░░░░░░░░░░░░░░░░░██░░██░░░░░░░░░░██    
#       ██░░░░░░░░░░░░░░░░░░██░░░░░░░░░░██░░░░░░░░░░░░░░░░▒▒▓▓░░██░░░░░░░░░░██    
#       ██░░░░░░░░▒▒░░░░░░░░██░░░░░░░░░░██░░░░░░░░░░░░░░░░██░░░░██░░░░░░░░░░██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██▒▒▓▓▓▓▓▓▓▓░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓██░░░░██▓▓▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▓▓▒▒▒▒██▓▓░░░░░░▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓▓▓▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒██▒▒▒▒██░░████████░░░░░░░░░░░░░░░░░░░░████████  ██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒████▒▒▒▒████████████░░░░░░░░░░░░░░░░░░░░████████████▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██  ██▒▒▒▒██  ████████░░░░░░░░░░░░░░░░░░░░████████  ██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██░░██▒▒▒▒██  ████████░░░░░░░░░░░░░░░░░░░░████████  ██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██░░██▒▒▒▒██▒▒▓▓  ░░██░░░░░░░░░░░░░░░░░░░░██░░░░██▒▒██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██░░██▒▒▒▒██▒▒▒▒████░░░░░░░░░░░░░░░░░░░░░░░░▓▓▓▓▒▒▒▒██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██░░██▒▒▒▒██▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▒▒██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▓▓██▒▒▒▒██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒██▒▒▒▒██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒██▒▒██████░░░░░░░░░░░░░░██░░░░░░░░░░░░░░░░██▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒██▒▒██▒▒▒▒██████░░░░░░░░░░░░░░░░░░░░██████▒▒▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒████▒▒▒▒▒▒▒▒██░░░░████████████████▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██▒▒▒▒▒▒██░░░░░░██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒████████░░░░░░██████▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▓▓░░    
#       ░░██▒▒▒▒██▒▒▒▒▒▒▒▒▒▒▓▓▓▓▓▓▓▓▓▓██░░░░░░██▓▓▓▓▓▓▓▓▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██    


In [9]:
#     ░░░░░░░░░░░░        ▒▒▓▓██▓▓▓▓▓▓▓▓▓▓██▓▓▓▓▓▓    ░░░░░░░░░░░░░░        
#           ░░░░░░░░░░░░  ██████░░░░░░░░░░░░░░░░  ░░░░░░████░░░░░░░░░░░░░░        
#           ░░░░░░░░░░████  ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░████░░░░░░░░          
#                   ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░    ██                
#               ░░██    ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██              
#             ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██          
#               ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██            
#             ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██          
#           ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓        
#           ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██        
#           ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██        
#         ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██      
#         ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░    ░░██      
#         ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░  ░░    ░░██      
#         ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██      
#       ▓▓░░░░░░████░░▓▓▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓░░░░░░░░░░▒▒▓▓    
#       ██░░▒▒▓▓▒▒▒▒██▒▒▒▒██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██░░░░░░░░░░░░██    
#       ██▒▒░░██▒▒▒▒▒▒▒▒▒▒██░░░░░░░░░░░░░░▓▓░░░░░░░░░░░░░░░░░░██░░░░░░░░░░░░██    
#       ██░░░░▒▒██▒▒▒▒▒▒██░░░░░░░░░░░░░░░░██░░░░░░░░░░░░░░░░░░██░░░░░░░░░░░░██    
#       ██░░░░░░░░██████▒▒░░░░░░░░░░░░░░██░░░░░░░░░░░░░░░░░░██░░██░░░░░░░░░░██    
#       ██░░░░░░░░░░░░░░░░░░██░░░░░░░░░░██░░░░░░░░░░░░░░░░▒▒▓▓░░██░░░░░░░░░░██    
#       ██░░░░░░░░▒▒░░░░░░░░██░░░░░░░░░░██░░░░░░░░░░░░░░░░██░░░░██░░░░░░░░░░██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██▒▒▓▓▓▓▓▓▓▓░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓██░░░░██▓▓▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▓▓▒▒▒▒██▓▓░░░░░░▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓▓▓▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒██▒▒▒▒██░░░█████░░░░░░░░░░░░░░░░░░░░░░████████  ██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒████▒▒▒▒█████░░░░░██░░░░░░░░░░░░░░░░░░░░███░░░░░░███▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██  ██▒▒▒▒██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░████████  ██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██░░██▒▒▒▒██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░████████  ██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██░░██▒▒▒▒██▒▒▓▓  ░░░░░░░░░░░░░░░░░░░░░░░░██░░░░██▒▒██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██░░██▒▒▒▒██▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓▓▓▒▒▒▒██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██░░██▒▒▒▒██▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▒▒██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▓▓██▒▒▒▒██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒██▒▒▒▒██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒██▒▒██████░░░░░░░░░░░░░░██░░░░░░░░░░░░░░░░██▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒██▒▒██▒▒▒▒██████░░░░░░░░░░░░░░░░░░░░██████▒▒▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒████▒▒▒▒▒▒▒▒██░░░░████████████████▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██▒▒▒▒▒▒██░░░░░░██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒████████░░░░░░██████▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▓▓░░    
#       ░░██▒▒▒▒██▒▒▒▒▒▒▒▒▒▒▓▓▓▓▓▓▓▓▓▓██░░░░░░██▓▓▓▓▓▓▓▓▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██    


In [10]:
░

SyntaxError: invalid character in identifier (<ipython-input-10-5096b85cee69>, line 1)

In [11]:
█

SyntaxError: invalid character in identifier (<ipython-input-11-cfac5b8ac397>, line 1)

In [2]:
import pickle
def save_obj(obj, name ):
    with open('./'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [3]:
def negative_downsample(df, target, neg_frac = .1):
    # Negative downsampling
    df_pos = df[df[target]==1]
    df_neg = df[df[target]==0]
    
    df_neg = df_neg.sample(int(df.shape[0] * neg_frac), random_state=42)
    return pd.concat([df_pos,df_neg]).sort_index()

In [7]:
dtypes = {'label': 'int8',
 'uid': 'int32',
 'task_id': 'int16',
 'adv_id': 'int16',
 'creat_type_cd': 'int8',
 'adv_prim_id': 'int16',
 'dev_id': 'int8',
 'inter_type_cd': 'int8',
 'slot_id': 'int8',
 'spread_app_id': 'int8',
 'tags': 'int8',
 'app_first_class': 'int8',
 'app_second_class': 'int8',
 'age': 'int8',
 'city': 'int16',
 'city_rank': 'int8',
 'device_name': 'int8',
 'device_size': 'int16',
 'career': 'int8',
 'gender': 'int8',
 'net_type': 'int8',
 'residence': 'int8',
 'his_app_size': 'int8',
 'his_on_shelf_time': 'int8',
 'app_score': 'int8',
 'emui_dev': 'int8',
 'list_time': 'int8',
 'device_price': 'int8',
 'up_life_duration': 'int8',
 'up_membership_grade': 'int8',
 'membership_life_duration': 'int8',
 'consume_purchase': 'int8',
 'communication_onlinerate': 'object',
 'communication_avgonline_30d': 'int8',
 'indu_name': 'int8',
 'pt_d': 'int8'}

In [8]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [34]:
# train_df  = pd.read_csv('train_data/train_data.csv', sep="|",dtype = dtypes ).reset_index()
# test_df = pd.read_csv('test_data_A.csv', sep="|",dtype = dtypes ).reset_index()



# b = pd.concat([train_df.drop('label',axis=1),test_df.drop('id',axis=1)]).sort_values(['pt_d','index']).reset_index()
# for col in ['task_id','adv_prim_id']:
#     conts = []
#     for i in range(2,9):
#         a = b[b.pt_d==i-1].groupby(['uid',col]).size().reset_index().rename({0:f'uid_{col}_ptd'},axis=1)
#         a['pt_d'] = i

#         conts.append(a)
#     c = pd.concat(conts)
#     train_df = pd.merge(train_df,c,how='left', on=['uid',col,'pt_d']).sort_values('index').reset_index(drop=True)
#     test_df = pd.merge(test_df,c,how='left', on=['uid',col,'pt_d']).sort_values('index').reset_index(drop=True)
    
#     gc.collect()

# for col in ['task_id','adv_id','slot_id','spread_app_id']:
#     conts = []
#     for i in range(2,9):
#         a = b[b.pt_d.isin(range(i))].groupby(['uid',col]).size().reset_index().rename({0:f'uid_{col}_ptd_rng'},axis=1)
#         a['pt_d'] = i

#         conts.append(a)
#     c = pd.concat(conts)
#     train_df = pd.merge(train_df,c,how='left', on=['uid',col,'pt_d']).sort_values('index').reset_index(drop=True)
#     test_df = pd.merge(test_df,c,how='left', on=['uid',col,'pt_d']).sort_values('index').reset_index(drop=True)
    
#     gc.collect()

# b = train_df.sort_values(['pt_d','index']).reset_index()


# for col in ['uid','task_id','adv_id']:
#     conts = []
#     for i in range(2,9):
#         a =  b[b.pt_d==i-1][[col,'label']].groupby([col]).mean().reset_index().rename({'label':f'{col}_targ_mean'},axis=1)
#         a['pt_d'] = i

#         conts.append(a)
#     c = pd.concat(conts)
#     train_df = pd.merge(train_df,c,how='left', on=[col,'pt_d']).sort_values('index').reset_index(drop=True)
#     test_df = pd.merge(test_df,c,how='left', on=[col,'pt_d']).sort_values('index').reset_index(drop=True)
#     gc.collect()




# for col in ['uid','task_id']:
#     conts = []
#     for i in range(2,9):
#         a =  b[b.pt_d.isin(range(i))][[col,'label']].groupby([col]).mean().reset_index().rename({'label':f'{col}_targ_mean_sumed'},axis=1)
#         a['pt_d'] = i

#         conts.append(a)
#     c = pd.concat(conts)
#     train_df = pd.merge(train_df,c,how='left', on=[col,'pt_d']).sort_values('index').reset_index(drop=True)
#     test_df = pd.merge(test_df,c,how='left', on=[col,'pt_d']).sort_values('index').reset_index(drop=True)
    
#     gc.collect()

# for col in ['task_id','adv_id']:
#     conts = []
#     for i in range(2,9):
#         a =  b[b.pt_d.isin(range(i))][['uid',col,'label']].groupby(['uid',col]).mean().reset_index().rename({'label':f'uid_{col}_targ_mean_sumed'},axis=1)
#         a['pt_d'] = i
#         conts.append(a)
#     c = pd.concat(conts)
#     train_df = pd.merge(train_df,c,how='left', on=['uid',col,'pt_d']).sort_values('index').reset_index(drop=True)
#     test_df = pd.merge(test_df,c,how='left', on=['uid',col,'pt_d']).sort_values('index').reset_index(drop=True)
    
#     gc.collect()

# for col in ['uid','task_id']:
#     conts = []
#     for i in range(2,9):
#         a =  b[b.pt_d.isin(range(i))][[col,'label']].groupby([col]).sum().reset_index().rename({'label':f'{col}_targ_sum_sumed'},axis=1)
#         a['pt_d'] = i

#         conts.append(a)
#     c = pd.concat(conts)
#     train_df = pd.merge(train_df,c,how='left', on=[col,'pt_d']).sort_values('index').reset_index(drop=True)
#     test_df = pd.merge(test_df,c,how='left', on=[col,'pt_d']).sort_values('index').reset_index(drop=True)
    
#     gc.collect()

# for col in ['task_id']:
#     conts = []
#     for i in range(2,9):
#         a =  b[b.pt_d.isin(range(i))][['career','age','gender',col,'label']].groupby(['career','age','gender',col]).mean().reset_index().rename({'label':f'career_age_gender_{col}_targ_mean_sumed'},axis=1)
#         a['pt_d'] = i
#         conts.append(a)
#     c = pd.concat(conts)
#     train_df = pd.merge(train_df,c,how='left', on=['career','age','gender',col,'pt_d']).sort_values('index').reset_index(drop=True)
#     test_df = pd.merge(test_df,c,how='left', on=['career','age','gender',col,'pt_d']).sort_values('index').reset_index(drop=True)
    
#     gc.collect()

# for col in ['uid','task_id']:
#     conts = []
#     for i in range(2,9):
#         a =  b[b.pt_d.isin(range(i))][[col,'label']].groupby([col]).std().reset_index().rename({'label':f'{col}_targ_mean_sumed'},axis=1)
#         a['pt_d'] = i

#         conts.append(a)
#     c = pd.concat(conts)
#     train_df = pd.merge(train_df,c,how='left', on=[col,'pt_d']).sort_values('index').reset_index(drop=True)
#     test_df = pd.merge(test_df,c,how='left', on=[col,'pt_d']).sort_values('index').reset_index(drop=True)
    
#     gc.collect()






# for col in ['uid','task_id','adv_id','slot_id','spread_app_id']:
#     conts = []
#     for i in range(2,9):
#         a = b[b.pt_d.isin(range(i))].groupby([col]).size().reset_index().rename({0:f'{col}_ptd_rng'},axis=1)
#         a['pt_d'] = i

#         conts.append(a)
#     c = pd.concat(conts)
#     train_df = pd.merge(train_df,c,how='left', on=[col,'pt_d']).sort_values('index').reset_index(drop=True)
#     test_df = pd.merge(test_df,c,how='left', on=[col,'pt_d']).sort_values('index').reset_index(drop=True)
    
#     gc.collect()
    





# groups  = pd.concat([train_df[['uid']],test_df[['uid']]]).groupby('uid').cumcount()

# train_df['uid_cumcount'] = groups[:len(train_df)]
# test_df['uid_cumcount'] = groups[len(train_df):]

# groups  = pd.concat([train_df[['uid','task_id']],test_df[['uid','task_id']]]).groupby(['uid','task_id']).cumcount()
# gc.collect()
# train_df['uid_task_id_cumcount'] = groups[:len(train_df)]
# test_df['uid_task_id_cumcount'] = groups[len(train_df):]

# groups  = pd.concat([train_df[['uid','spread_app_id']],test_df[['uid','spread_app_id']]]).groupby(['uid','spread_app_id']).cumcount()
# gc.collect()
# train_df['uid_spread_app_id_cumcount'] = groups[:len(train_df)]
# test_df['uid_spread_app_id_cumcount'] = groups[len(train_df):]


# for col in ['task_id','spread_app_id']:
#     groups  = pd.concat([train_df[[col]],test_df[[col]]]).groupby(col).cumcount()
#     gc.collect()
#     train_df[f'{col}_cumcount'] = groups[:len(train_df)]
#     test_df[f'{col}_cumcount'] = groups[len(train_df):]

# save_obj(test_df,'test_preproc')
# save_obj(train_df,'train_preproc')

train_df= load_obj('train_preproc.pkl').sort_values(['pt_d','index']).pipe(reduce_mem_usage)
test_df = load_obj('test_preproc.pkl').sort_values(['pt_d','index']).pipe(reduce_mem_usage)
b = train_df.sort_values(['pt_d','index']).reset_index()
for col in ['task_id']:
    conts = []
    for i in range(2,9):
        a =  b[b.pt_d.isin(range(i))][['uid_cumcount',col,'label']].groupby(['uid_cumcount',col]).mean().reset_index().rename({'label':f'uid_cumcount_{col}_targ_mean_sumed'},axis=1)
        a['pt_d'] = i
        conts.append(a)
    c = pd.concat(conts)
    train_df = pd.merge(train_df,c,how='left', on=['uid_cumcount',col,'pt_d']).sort_values('index').reset_index(drop=True)
    test_df = pd.merge(test_df,c,how='left', on=['uid_cumcount',col,'pt_d']).sort_values('index').reset_index(drop=True)
    
    gc.collect()

for col in ['task_id']:
    conts = []
    for i in range(2,9):
        a =  b[b.pt_d.isin(range(i))][['age',col,'label']].groupby(['age',col]).mean().reset_index().rename({'label':f'age_{col}_targ_mean_sumed'},axis=1)
        a['pt_d'] = i
        conts.append(a)
    c = pd.concat(conts)
    train_df = pd.merge(train_df,c,how='left', on=['age',col,'pt_d']).sort_values('index').reset_index(drop=True)
    test_df = pd.merge(test_df,c,how='left', on=['age',col,'pt_d']).sort_values('index').reset_index(drop=True)
    
    gc.collect()


for col in [ 'city',
       'city_rank', 'device_name', 'device_size', 'career', 'gender',
        'communication_onlinerate',
       'communication_avgonline_30d', 'indu_name']:
    conts = []
    for i in range(2,9):
        a =  b[b.pt_d.isin(range(i))][[col,'task_id','label']].groupby([col,'task_id']).mean().reset_index().rename({'label':f'{col}_task_id_targ_mean_sumed'},axis=1)
        a['pt_d'] = i
        conts.append(a)
    c = pd.concat(conts)
    train_df = pd.merge(train_df,c,how='left', on=[col,'task_id','pt_d']).sort_values('index').reset_index(drop=True).pipe(reduce_mem_usage)
    test_df = pd.merge(test_df,c,how='left', on=[col,'task_id','pt_d']).sort_values('index').reset_index(drop=True).pipe(reduce_mem_usage)
    
    gc.collect()

    
cnt = CountEncoder(cols=['uid','task_id','adv_id','creat_type_cd','adv_prim_id','dev_id','slot_id','spread_app_id','indu_name'])

cnt.fit(pd.concat([train_df.drop('label',axis =1),test_df.drop(['id'],axis=1)]))

train_m = cnt.transform(train_df.drop('label',axis=1))
test_m = cnt.transform(test_df.drop('id',axis=1))

Mem. usage decreased to 5315.45 Mb (14.2% reduction)
Mem. usage decreased to 129.70 Mb (13.9% reduction)
Mem. usage decreased to 5395.38 Mb (4.3% reduction)
Mem. usage decreased to 131.61 Mb (4.2% reduction)
Mem. usage decreased to 5475.31 Mb (4.2% reduction)
Mem. usage decreased to 133.51 Mb (4.1% reduction)
Mem. usage decreased to 5555.24 Mb (4.1% reduction)
Mem. usage decreased to 135.42 Mb (4.1% reduction)
Mem. usage decreased to 5635.17 Mb (4.1% reduction)
Mem. usage decreased to 137.33 Mb (4.0% reduction)
Mem. usage decreased to 5715.10 Mb (4.0% reduction)
Mem. usage decreased to 139.24 Mb (3.9% reduction)
Mem. usage decreased to 5795.03 Mb (4.0% reduction)
Mem. usage decreased to 141.14 Mb (3.9% reduction)
Mem. usage decreased to 5874.97 Mb (3.9% reduction)
Mem. usage decreased to 143.05 Mb (3.8% reduction)
Mem. usage decreased to 5954.90 Mb (3.9% reduction)
Mem. usage decreased to 144.96 Mb (3.8% reduction)
Mem. usage decreased to 6034.83 Mb (3.8% reduction)
Mem. usage decrease

In [35]:
test_df.columns

Index(['index', 'id', 'uid', 'task_id', 'adv_id', 'creat_type_cd',
       'adv_prim_id', 'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id',
       'tags', 'app_first_class', 'app_second_class', 'age', 'city',
       'city_rank', 'device_name', 'device_size', 'career', 'gender',
       'net_type', 'residence', 'his_app_size', 'his_on_shelf_time',
       'app_score', 'emui_dev', 'list_time', 'device_price',
       'up_life_duration', 'up_membership_grade', 'membership_life_duration',
       'consume_purchase', 'communication_onlinerate',
       'communication_avgonline_30d', 'indu_name', 'pt_d', 'uid_task_id_ptd',
       'uid_adv_prim_id_ptd', 'uid_task_id_ptd_rng', 'uid_adv_id_ptd_rng',
       'uid_slot_id_ptd_rng', 'uid_spread_app_id_ptd_rng', 'uid_targ_mean',
       'task_id_targ_mean', 'adv_id_targ_mean', 'uid_targ_mean_sumed_x',
       'task_id_targ_mean_sumed_x', 'uid_task_id_targ_mean_sumed',
       'uid_adv_id_targ_mean_sumed', 'uid_targ_sum_sumed',
       'task_id_targ_sum_s

In [37]:
train_m['label'] = train_df.label

In [38]:
train_m[35897957:].pt_d.value_counts()

7    6009176
Name: pt_d, dtype: int64

In [39]:
train_m[:6277359].pt_d.value_counts()

1    6277359
Name: pt_d, dtype: int64

In [47]:
FEATS = ['uid','task_id','adv_id','creat_type_cd','adv_prim_id','dev_id','slot_id','spread_app_id','indu_name','age','uid_task_id_ptd','uid_adv_prim_id_ptd',
        'uid_task_id_ptd','uid_adv_id_ptd_rng','uid_slot_id_ptd_rng','uid_spread_app_id_ptd_rng','his_app_size','app_first_class','gender',
        'career','device_name','uid_targ_mean','task_id_targ_mean','adv_id_targ_mean','uid_targ_mean_sumed_x',
        'task_id_targ_mean_sumed_x','uid_task_id_targ_mean_sumed','uid_adv_id_targ_mean_sumed','uid_targ_sum_sumed','task_id_targ_sum_sumed'
         ,'career_age_gender_task_id_targ_mean_sumed', 'uid_ptd_rng','app_first_class','consume_purchase','up_membership_grade','up_life_duration',
        'list_time','uid_cumcount','uid_task_id_cumcount','uid_spread_app_id_cumcount','uid_cumcount_task_id_targ_mean_sumed','age_task_id_targ_mean_sumed',
         'city_task_id_targ_mean_sumed', 'city_rank_task_id_targ_mean_sumed','device_name_task_id_targ_mean_sumed',
         'device_size_task_id_targ_mean_sumed', 'career_task_id_targ_mean_sumed','gender_task_id_targ_mean_sumed',
         'communication_onlinerate_task_id_targ_mean_sumed',
       'communication_avgonline_30d_task_id_targ_mean_sumed',
       'indu_name_task_id_targ_mean_sumed'
         
        ]
PARAMS = {  'learning_rate':0.05, 'max_depth': -1,
        'n_estimators': 200,
        'num_leaves': 250,'objective': 'binary', 'n_jobs': -1,'boost_from_average': False}


train_downded = negative_downsample(train_m[6277359:35897957],'label')
train_data = lgb.Dataset(train_downded.drop('label',axis=1)[FEATS], label=train_downded.label, free_raw_data=True)
aa = lgb.train(PARAMS,train_data)
preds = aa.predict(train_m[35897957:].drop('label',axis=1)[FEATS])
roc_auc_score(train_m[35897957:].label,preds)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.834216807637092

In [41]:
FEATS = ['uid','task_id','adv_id','creat_type_cd','adv_prim_id','dev_id','slot_id','spread_app_id','indu_name','age','uid_task_id_ptd','uid_adv_prim_id_ptd',
        'uid_task_id_ptd','uid_adv_id_ptd_rng','uid_slot_id_ptd_rng','uid_spread_app_id_ptd_rng','his_app_size','app_first_class','gender',
        'career','device_name','uid_targ_mean','task_id_targ_mean','adv_id_targ_mean','uid_targ_mean_sumed_x',
        'task_id_targ_mean_sumed_x','uid_task_id_targ_mean_sumed','uid_adv_id_targ_mean_sumed','uid_targ_sum_sumed','task_id_targ_sum_sumed'
         ,'career_age_gender_task_id_targ_mean_sumed', 'uid_ptd_rng','app_first_class','consume_purchase','up_membership_grade','up_life_duration',
        'list_time','uid_cumcount','uid_task_id_cumcount','uid_spread_app_id_cumcount','uid_cumcount_task_id_targ_mean_sumed','age_task_id_targ_mean_sumed',
         'city_task_id_targ_mean_sumed', 'city_rank_task_id_targ_mean_sumed','device_name_task_id_targ_mean_sumed',
         'device_size_task_id_targ_mean_sumed', 'career_task_id_targ_mean_sumed','gender_task_id_targ_mean_sumed'
         
         
        ]
PARAMS = {  'learning_rate':0.05, 'max_depth': -1,
        'n_estimators': 200,
        'num_leaves': 250,'objective': 'binary', 'n_jobs': -1,'boost_from_average': False}


train_downded = negative_downsample(train_m[6277359:35897957],'label')
train_data = lgb.Dataset(train_downded.drop('label',axis=1)[FEATS], label=train_downded.label, free_raw_data=True)
aa = lgb.train(PARAMS,train_data)
preds = aa.predict(train_m[35897957:].drop('label',axis=1)[FEATS])
roc_auc_score(train_m[35897957:].label,preds)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.8342143300039557

In [19]:



train_downded = negative_downsample(train_m[6277359:],'label')
train_data = lgb.Dataset(train_downded.drop('label',axis=1)[FEATS], label=train_downded.label, free_raw_data=True)
aa = lgb.train(PARAMS,train_data)
preds = aa.predict(test_m[FEATS])


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [20]:
test_df['probability']= preds

In [21]:
test_df[['id','probability']].set_index('id').to_csv('0.8303465728762043.csv')